In [1]:
import tushare as ts
ts.set_token('57b81c613f14d6d45c832968542884a0fe97b8e2fcda6f1fc67d17c3')
pro = ts.pro_api()

In [2]:
import pandas as pd
import os
import joblib

In [3]:
stock_basic = pd.read_csv("stock_basic.csv")
stock_basic

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,1,平安银行,深圳,银行,19910403
1,000002.SZ,2,万科A,深圳,全国地产,19910129
2,000004.SZ,4,国华网安,深圳,软件服务,19910114
3,000006.SZ,6,深振业A,深圳,区域地产,19920427
4,000007.SZ,7,全新好,深圳,其他商业,19920413
...,...,...,...,...,...,...
4530,605580.SH,605580,恒盛能源,浙江,供气供热,20210819
4531,605588.SH,605588,冠石科技,江苏,元器件,20210812
4532,605589.SH,605589,圣泉集团,山东,化工原料,20210810
4533,605598.SH,605598,上海港湾,上海,建筑工程,20210917


In [4]:
trade_cal = pd.read_csv("trade_cal.csv")
trade_cal

,exchange,cal_date,is_open,pretrade_date
0,SSE,20141027,1,20141024
1,SSE,20141028,1,20141027
2,SSE,20141029,1,20141028
3,SSE,20141030,1,20141029
4,SSE,20141031,1,20141030
...,...,...,...,...
2427,SSE,20241021,1,20241018
2428,SSE,20241022,1,20241021
2429,SSE,20241023,1,20241022
2430,SSE,20241024,1,20241023


In [5]:
custom_date = pd.read_csv("custom_date.csv")
start_date=str(custom_date.start_date[0])
end_date=str(custom_date.end_date[0])
print(start_date)
print(end_date)

20141027
20241027


In [7]:
df = pro.index_daily(ts_code='399905.SZ', start_date=start_date, end_date=end_date)
df["future_5"]= df.close.pct_change(-5).shift(5)
df

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount,future_5
0,399905.SZ,20241025,5809.6879,5735.2853,5855.2647,5729.4384,5726.7041,82.9838,1.4491,236725708.0,2.995902e+08,NaN
1,399905.SZ,20241024,5726.7041,5747.5577,5759.8651,5694.7789,5776.3830,-49.6789,-0.8600,207690046.0,2.572323e+08,NaN
2,399905.SZ,20241023,5776.3830,5745.1069,5853.4555,5727.7736,5753.8497,22.5333,0.3916,273412640.0,3.361677e+08,NaN
3,399905.SZ,20241022,5753.8497,5703.3196,5779.2983,5673.7901,5704.2250,49.6247,0.8700,237023624.0,3.125843e+08,NaN
4,399905.SZ,20241021,5704.2250,5680.5648,5797.8808,5631.2316,5640.0198,64.2052,1.1384,266885186.0,3.599657e+08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2427,399905.SZ,20141031,4986.4160,4999.0030,5012.2600,4945.2890,4992.6300,-6.2140,-0.1245,91669059.0,9.218221e+07,-0.001837
2428,399905.SZ,20141030,4992.6300,4988.9720,5021.1900,4975.7860,4991.7550,0.8750,0.0175,99168604.0,9.848613e+07,0.006453
2429,399905.SZ,20141029,4991.7550,4938.7150,4999.9580,4923.5150,4924.9550,66.8000,1.3564,105608671.0,1.064160e+08,0.000739
2430,399905.SZ,20141028,4924.9550,4819.5500,4925.0850,4819.5500,4809.5180,115.4370,2.4002,75113014.0,7.649555e+07,0.015942


In [8]:
df.to_csv("zz500.csv",index=0)

In [6]:
if not os.path.isdir("daily_basic"):
    os.makedirs("daily_basic")
    print("已创建daily_basic文件夹")
else:
    print("daily_basic文件夹已存在")

daily_basic文件夹已存在


In [7]:
#每日指标
all_index = stock_basic.shape[0]
df_index = pd.to_datetime(trade_cal.cal_date.tolist(),format='%Y%m%d')
for index, row in stock_basic.iterrows():
    stock_name = row.ts_code
    print(index, all_index, stock_name)
    current_df = pro.daily_basic(ts_code=stock_name,start_date=start_date,end_date=end_date)
    current_df.sort_values(by='trade_date',ascending=True,inplace=True)
    current_df.reset_index(inplace=True,drop=True)
    current_df.trade_date=pd.to_datetime(current_df.trade_date.tolist(),format='%Y%m%d')
    current_df.index=current_df.trade_date
    standard_df = pd.DataFrame(index=df_index,columns=current_df.columns.tolist())
    for column in current_df.columns:
        standard_df[column]=current_df[column]
    standard_df.trade_date=standard_df.index.tolist()
    standard_df.ts_code = stock_name
    standard_df.to_csv("daily_basic/"+stock_name+".csv",index=0)

0 4535 000001.SZ
1 4535 000002.SZ
2 4535 000004.SZ
3 4535 000006.SZ
4 4535 000007.SZ
5 4535 000008.SZ
6 4535 000009.SZ
7 4535 000010.SZ
8 4535 000011.SZ
9 4535 000012.SZ
10 4535 000014.SZ
11 4535 000016.SZ
12 4535 000017.SZ
13 4535 000019.SZ
14 4535 000020.SZ
15 4535 000021.SZ
16 4535 000025.SZ
17 4535 000026.SZ
18 4535 000027.SZ
19 4535 000028.SZ
20 4535 000029.SZ
21 4535 000030.SZ
22 4535 000031.SZ
23 4535 000032.SZ
24 4535 000034.SZ
25 4535 000035.SZ
26 4535 000036.SZ
27 4535 000037.SZ
28 4535 000039.SZ
29 4535 000040.SZ
30 4535 000042.SZ
31 4535 000045.SZ
32 4535 000048.SZ
33 4535 000049.SZ
34 4535 000050.SZ
35 4535 000055.SZ
36 4535 000056.SZ
37 4535 000058.SZ
38 4535 000059.SZ
39 4535 000060.SZ
40 4535 000061.SZ
41 4535 000062.SZ
42 4535 000063.SZ
43 4535 000065.SZ
44 4535 000066.SZ
45 4535 000068.SZ
46 4535 000069.SZ
47 4535 000070.SZ
48 4535 000078.SZ
49 4535 000088.SZ
50 4535 000089.SZ
51 4535 000090.SZ
52 4535 000096.SZ
53 4535 000099.SZ
54 4535 000100.SZ
55 4535 000151.SZ
56

437 4535 001223.SZ
438 4535 001225.SZ
439 4535 001226.SZ
440 4535 001227.SZ
441 4535 001228.SZ
442 4535 001229.SZ
443 4535 001230.SZ
444 4535 001231.SZ
445 4535 001234.SZ
446 4535 001236.SZ
447 4535 001238.SZ
448 4535 001239.SZ
449 4535 001255.SZ
450 4535 001256.SZ
451 4535 001258.SZ
452 4535 001259.SZ
453 4535 001260.SZ
454 4535 001266.SZ
455 4535 001267.SZ
456 4535 001268.SZ
457 4535 001269.SZ
458 4535 001270.SZ
459 4535 001277.SZ
460 4535 001278.SZ
461 4535 001279.SZ
462 4535 001282.SZ
463 4535 001283.SZ
464 4535 001286.SZ
465 4535 001287.SZ
466 4535 001288.SZ
467 4535 001289.SZ
468 4535 001296.SZ
469 4535 001298.SZ
470 4535 001299.SZ
471 4535 001300.SZ
472 4535 001301.SZ
473 4535 001306.SZ
474 4535 001308.SZ
475 4535 001309.SZ
476 4535 001311.SZ
477 4535 001313.SZ
478 4535 001314.SZ
479 4535 001316.SZ
480 4535 001317.SZ
481 4535 001318.SZ
482 4535 001319.SZ
483 4535 001322.SZ
484 4535 001323.SZ
485 4535 001324.SZ
486 4535 001326.SZ
487 4535 001328.SZ
488 4535 001330.SZ
489 4535 001

869 4535 002375.SZ
870 4535 002376.SZ
871 4535 002377.SZ
872 4535 002378.SZ
873 4535 002379.SZ
874 4535 002380.SZ
875 4535 002381.SZ
876 4535 002382.SZ
877 4535 002383.SZ
878 4535 002384.SZ
879 4535 002385.SZ
880 4535 002386.SZ
881 4535 002387.SZ
882 4535 002388.SZ
883 4535 002389.SZ
884 4535 002390.SZ
885 4535 002391.SZ
886 4535 002392.SZ
887 4535 002393.SZ
888 4535 002394.SZ
889 4535 002395.SZ
890 4535 002396.SZ
891 4535 002397.SZ
892 4535 002398.SZ
893 4535 002399.SZ
894 4535 002400.SZ
895 4535 002401.SZ
896 4535 002402.SZ
897 4535 002403.SZ
898 4535 002404.SZ
899 4535 002405.SZ
900 4535 002406.SZ
901 4535 002407.SZ
902 4535 002408.SZ
903 4535 002409.SZ
904 4535 002410.SZ
905 4535 002412.SZ
906 4535 002413.SZ
907 4535 002414.SZ
908 4535 002415.SZ
909 4535 002416.SZ
910 4535 002418.SZ
911 4535 002419.SZ
912 4535 002420.SZ
913 4535 002421.SZ
914 4535 002422.SZ
915 4535 002423.SZ
916 4535 002424.SZ
917 4535 002425.SZ
918 4535 002426.SZ
919 4535 002427.SZ
920 4535 002428.SZ
921 4535 002

1286 4535 002833.SZ
1287 4535 002835.SZ
1288 4535 002836.SZ
1289 4535 002837.SZ
1290 4535 002838.SZ
1291 4535 002839.SZ
1292 4535 002840.SZ
1293 4535 002841.SZ
1294 4535 002842.SZ
1295 4535 002843.SZ
1296 4535 002845.SZ
1297 4535 002846.SZ
1298 4535 002847.SZ
1299 4535 002848.SZ
1300 4535 002849.SZ
1301 4535 002850.SZ
1302 4535 002851.SZ
1303 4535 002852.SZ
1304 4535 002853.SZ
1305 4535 002855.SZ
1306 4535 002856.SZ
1307 4535 002857.SZ
1308 4535 002858.SZ
1309 4535 002859.SZ
1310 4535 002860.SZ
1311 4535 002861.SZ
1312 4535 002862.SZ
1313 4535 002863.SZ
1314 4535 002864.SZ
1315 4535 002865.SZ
1316 4535 002866.SZ
1317 4535 002867.SZ
1318 4535 002868.SZ
1319 4535 002869.SZ
1320 4535 002870.SZ
1321 4535 002871.SZ
1322 4535 002872.SZ
1323 4535 002873.SZ
1324 4535 002875.SZ
1325 4535 002876.SZ
1326 4535 002877.SZ
1327 4535 002878.SZ
1328 4535 002879.SZ
1329 4535 002880.SZ
1330 4535 002881.SZ
1331 4535 002882.SZ
1332 4535 002883.SZ
1333 4535 002884.SZ
1334 4535 002885.SZ
1335 4535 002886.SZ


1696 4535 300230.SZ
1697 4535 300231.SZ
1698 4535 300232.SZ
1699 4535 300233.SZ
1700 4535 300234.SZ
1701 4535 300235.SZ
1702 4535 300236.SZ
1703 4535 300237.SZ
1704 4535 300238.SZ
1705 4535 300239.SZ
1706 4535 300240.SZ
1707 4535 300241.SZ
1708 4535 300242.SZ
1709 4535 300243.SZ
1710 4535 300244.SZ
1711 4535 300245.SZ
1712 4535 300246.SZ
1713 4535 300247.SZ
1714 4535 300248.SZ
1715 4535 300249.SZ
1716 4535 300250.SZ
1717 4535 300251.SZ
1718 4535 300252.SZ
1719 4535 300253.SZ
1720 4535 300254.SZ
1721 4535 300255.SZ
1722 4535 300256.SZ
1723 4535 300257.SZ
1724 4535 300258.SZ
1725 4535 300259.SZ
1726 4535 300260.SZ
1727 4535 300261.SZ
1728 4535 300263.SZ
1729 4535 300264.SZ
1730 4535 300265.SZ
1731 4535 300266.SZ
1732 4535 300267.SZ
1733 4535 300268.SZ
1734 4535 300269.SZ
1735 4535 300270.SZ
1736 4535 300271.SZ
1737 4535 300272.SZ
1738 4535 300274.SZ
1739 4535 300275.SZ
1740 4535 300276.SZ
1741 4535 300277.SZ
1742 4535 300278.SZ
1743 4535 300279.SZ
1744 4535 300280.SZ
1745 4535 300281.SZ


...

2516 4535 301097.SZ
2517 4535 301098.SZ
2518 4535 301099.SZ
2519 4535 301100.SZ
2520 4535 301101.SZ
2521 4535 301102.SZ
2522 4535 301103.SZ
2523 4535 301105.SZ
2524 4535 301106.SZ
2525 4535 301107.SZ
2526 4535 301108.SZ
2527 4535 301109.SZ
2528 4535 301110.SZ
2529 4535 301111.SZ
2530 4535 301112.SZ
2531 4535 301113.SZ
2532 4535 301115.SZ
2533 4535 301116.SZ
2534 4535 301117.SZ
2535 4535 301118.SZ
2536 4535 301119.SZ
2537 4535 301120.SZ
2538 4535 301121.SZ
2539 4535 301122.SZ
2540 4535 301123.SZ
2541 4535 301125.SZ
2542 4535 301126.SZ
2543 4535 301127.SZ
2544 4535 301128.SZ
2545 4535 301129.SZ
2546 4535 301130.SZ
2547 4535 301131.SZ
2548 4535 301132.SZ
2549 4535 301133.SZ
2550 4535 301135.SZ
2551 4535 301136.SZ
2552 4535 301137.SZ
2553 4535 301138.SZ
2554 4535 301139.SZ
2555 4535 301141.SZ
2556 4535 301148.SZ
2557 4535 301149.SZ
2558 4535 301150.SZ
2559 4535 301151.SZ
2560 4535 301152.SZ
2561 4535 301153.SZ
2562 4535 301155.SZ
2563 4535 301156.SZ
2564 4535 301157.SZ
2565 4535 301158.SZ


2926 4535 600113.SH
2927 4535 600114.SH
2928 4535 600115.SH
2929 4535 600116.SH
2930 4535 600117.SH
2931 4535 600118.SH
2932 4535 600119.SH
2933 4535 600120.SH
2934 4535 600121.SH
2935 4535 600123.SH
2936 4535 600125.SH
2937 4535 600126.SH
2938 4535 600127.SH
2939 4535 600128.SH
2940 4535 600129.SH
2941 4535 600130.SH
2942 4535 600131.SH
2943 4535 600132.SH
2944 4535 600133.SH
2945 4535 600135.SH
2946 4535 600136.SH
2947 4535 600137.SH
2948 4535 600138.SH
2949 4535 600141.SH
2950 4535 600143.SH
2951 4535 600148.SH
2952 4535 600149.SH
2953 4535 600150.SH
2954 4535 600151.SH
2955 4535 600152.SH
2956 4535 600153.SH
2957 4535 600155.SH
2958 4535 600156.SH
2959 4535 600157.SH
2960 4535 600158.SH
2961 4535 600159.SH
2962 4535 600160.SH
2963 4535 600161.SH
2964 4535 600162.SH
2965 4535 600163.SH
2966 4535 600165.SH
2967 4535 600166.SH
2968 4535 600167.SH
2969 4535 600168.SH
2970 4535 600169.SH
2971 4535 600170.SH
2972 4535 600171.SH
2973 4535 600172.SH
2974 4535 600173.SH
2975 4535 600176.SH


3336 4535 600648.SH
3337 4535 600649.SH
3338 4535 600650.SH
3339 4535 600651.SH
3340 4535 600653.SH
3341 4535 600654.SH
3342 4535 600655.SH
3343 4535 600657.SH
3344 4535 600658.SH
3345 4535 600660.SH
3346 4535 600661.SH
3347 4535 600662.SH
3348 4535 600663.SH
3349 4535 600664.SH
3350 4535 600665.SH
3351 4535 600666.SH
3352 4535 600667.SH
3353 4535 600668.SH
3354 4535 600671.SH
3355 4535 600673.SH
3356 4535 600674.SH
3357 4535 600675.SH
3358 4535 600676.SH
3359 4535 600678.SH
3360 4535 600679.SH
3361 4535 600681.SH
3362 4535 600682.SH
3363 4535 600683.SH
3364 4535 600684.SH
3365 4535 600685.SH
3366 4535 600686.SH
3367 4535 600688.SH
3368 4535 600689.SH
3369 4535 600690.SH
3370 4535 600691.SH
3371 4535 600692.SH
3372 4535 600693.SH
3373 4535 600694.SH
3374 4535 600696.SH
3375 4535 600697.SH
3376 4535 600698.SH
3377 4535 600699.SH
3378 4535 600702.SH
3379 4535 600703.SH
3380 4535 600704.SH
3381 4535 600705.SH
3382 4535 600706.SH
3383 4535 600707.SH
3384 4535 600708.SH
3385 4535 600710.SH


3746 4535 601636.SH
3747 4535 601658.SH
3748 4535 601665.SH
3749 4535 601666.SH
3750 4535 601668.SH
3751 4535 601669.SH
3752 4535 601677.SH
3753 4535 601678.SH
3754 4535 601686.SH
3755 4535 601688.SH
3756 4535 601689.SH
3757 4535 601696.SH
3758 4535 601698.SH
3759 4535 601699.SH
3760 4535 601700.SH
3761 4535 601702.SH
3762 4535 601717.SH
3763 4535 601718.SH
3764 4535 601727.SH
3765 4535 601728.SH
3766 4535 601766.SH
3767 4535 601777.SH
3768 4535 601778.SH
3769 4535 601788.SH
3770 4535 601789.SH
3771 4535 601798.SH
3772 4535 601799.SH
3773 4535 601800.SH
3774 4535 601801.SH
3775 4535 601808.SH
3776 4535 601811.SH
3777 4535 601816.SH
3778 4535 601818.SH
3779 4535 601825.SH
3780 4535 601827.SH
3781 4535 601828.SH
3782 4535 601838.SH
3783 4535 601857.SH
3784 4535 601858.SH
3785 4535 601860.SH
3786 4535 601865.SH
3787 4535 601866.SH
3788 4535 601868.SH
3789 4535 601869.SH
3790 4535 601872.SH
3791 4535 601877.SH
3792 4535 601878.SH
3793 4535 601880.SH
3794 4535 601881.SH
3795 4535 601882.SH


4156 4535 603506.SH
4157 4535 603507.SH
4158 4535 603508.SH
4159 4535 603511.SH
4160 4535 603515.SH
4161 4535 603516.SH
4162 4535 603517.SH
4163 4535 603518.SH
4164 4535 603519.SH
4165 4535 603520.SH
4166 4535 603527.SH
4167 4535 603528.SH
4168 4535 603529.SH
4169 4535 603530.SH
4170 4535 603533.SH
4171 4535 603535.SH
4172 4535 603536.SH
4173 4535 603538.SH
4174 4535 603551.SH
4175 4535 603556.SH
4176 4535 603557.SH
4177 4535 603558.SH
4178 4535 603559.SH
4179 4535 603565.SH
4180 4535 603566.SH
4181 4535 603567.SH
4182 4535 603568.SH
4183 4535 603569.SH
4184 4535 603577.SH
4185 4535 603578.SH
4186 4535 603579.SH
4187 4535 603580.SH
4188 4535 603583.SH
4189 4535 603585.SH
4190 4535 603586.SH
4191 4535 603587.SH
4192 4535 603588.SH
4193 4535 603589.SH
4194 4535 603590.SH
4195 4535 603595.SH
4196 4535 603596.SH
4197 4535 603598.SH
4198 4535 603599.SH
4199 4535 603600.SH
4200 4535 603601.SH
4201 4535 603602.SH
4202 4535 603605.SH
4203 4535 603606.SH
4204 4535 603607.SH
4205 4535 603608.SH


In [8]:
# standard_df